### Dash Lab

In [20]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
import plotly.express as px
from dash.dependencies import Input, Output

spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
spacex_df.drop(columns = {"Unnamed: 0"}, inplace = True)

# Calculate the success rates for each Launch Site
success = spacex_df.groupby("Launch Site").agg({"class": "sum"}).reset_index()
success["percentage"] = success["class"] / success["class"].sum()

# Create the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard"),

    # TASK 1: Launch Site Drop-down Input Component
    dcc.Dropdown(
        id = "launch-site-dropdown",
        options = [{"label": site, "value": site} for site in success["Launch Site"]],
        value = success["Launch Site"].iloc[0],  # Default value for the dropdown
    ),
    
    # TASK 2: Callback to render success-pie-chart based on selected site dropdown
    html.Div([
        html.Label("Success Rate for Each Launch Site:"),
        dcc.Graph(id = "success-pie-chart"),
    ]),

    # TASK 3: Range Slider to Select Payload
    dcc.RangeSlider(
        id = "payload-range-slider",
        min = int(spacex_df["Payload Mass (kg)"].min()),  # Convert to integer
        max = int(spacex_df["Payload Mass (kg)"].max()),  # Convert to integer
        step = 1000,
        value = [int(spacex_df["Payload Mass (kg)"].min()), int(spacex_df["Payload Mass (kg)"].max())],  # Convert to integer
        marks = {payload: str(payload) for payload in range(
            int(spacex_df["Payload Mass (kg)"].min()),
            int(spacex_df["Payload Mass (kg)"].max()) + 1,
            5000
        )},
    ),

    # TASK 4: Callback to render the success-payload-scatter-chart scatter plot
    html.Div([
        html.Label("Scatter Plot: Payload Mass (kg) vs. Class"),
        dcc.Graph(id = "success-payload-scatter-chart"),
    ]),
])

# TASK 2: Callback function to update pie chart based on selected launch site
@app.callback(
    Output("success-pie-chart", "figure"),
    [Input("launch-site-dropdown", "value")]
)
def update_pie_chart(selected_site):
    pie_chart_fig = go.Figure(
        data = go.Pie(
            labels = success["Launch Site"],
            values = success["percentage"],
            hole = 0.3,
        ),
    )
    pie_chart_fig.update_layout(title_text = "Success Rate by Launch Site")
    return pie_chart_fig

# TASK 4: Callback function to update scatter plot based on selected payload range
@app.callback(
    Output("success-payload-scatter-chart", "figure"),
    [Input("payload-range-slider", "value")]
)
def update_scatter_chart(selected_payload_range):
    filtered_data = spacex_df[
        (spacex_df["Payload Mass (kg)"] >= selected_payload_range[0]) &
        (spacex_df["Payload Mass (kg)"] <= selected_payload_range[1])
    ]
    scatter_fig = px.scatter(
        filtered_data,
        x = "Payload Mass (kg)",
        y = "class",
        color = "Launch Site",
        color_discrete_sequence=px.colors.qualitative.Set1,  # Set consistent colors for launch sites
        title = "Payload Mass vs. Class",
    )
    return scatter_fig

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)